In [1]:
%matplotlib inline

In [13]:
path = "/home/ubuntu/nbs/data/dogscats.3/"
model = "lesson1-diy-weights-2.h5"
predictions = "lesson1-diy-pred.npz"



In [3]:
from __future__ import division,print_function

import os, json
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt
import utils; reload(utils)
from utils import plots
import vgg16; reload(vgg16)
from vgg16 import Vgg16
import re

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [8]:
if os.path.isfile(model):
     batch_size=4
     vgg = Vgg16()
     vgg.model.load_weights(model)
     vgg.classes = ["cat", "dog"]   
else:
     batch_size=64
     vgg = Vgg16()    
     batches = vgg.get_batches(path+'train', batch_size=batch_size)
     val_batches = vgg.get_batches(path+'valid', batch_size=batch_size*2)
     vgg.finetune(batches)
     vgg.model.optimizer.lr = 0.01
     vgg.fit(batches, val_batches, nb_epoch=5)
     vgg.model.save_weights(model)


Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Epoch 1/5
23000/23000 [==============================] - 598s - loss: 0.3590 - acc: 0.9690 - val_loss: 0.3300 - val_acc: 0.9725
Epoch 2/5
23000/23000 [==============================] - 598s - loss: 0.3440 - acc: 0.9747 - val_loss: 0.2746 - val_acc: 0.9795
Epoch 3/5
23000/23000 [==============================] - 598s - loss: 0.3290 - acc: 0.9768 - val_loss: 0.2505 - val_acc: 0.9815
Epoch 4/5
23000/23000 [==============================] - 598s - loss: 0.3496 - acc: 0.9761 - val_loss: 0.2941 - val_acc: 0.9790
Epoch 5/5
23000/23000 [==============================] - 600s - loss: 0.3176 - acc: 0.9782 - val_loss: 0.3446 - val_acc: 0.9765


In [19]:
try:
    np.load(predictions)
except:
    pred_batches,pred_gen = vgg.test(path+'test', batch_size=batch_size) 
    ids = map(lambda fn: int(re.sub(r"\D","", fn)),pred_batches.filenames)
    np.savez(predictions, pred_gen, ids)

In [33]:
dogs = np.clip(pred_gen[:,1], 0.015, 0.985)
#preds = [ abs(pred_gen[i, idxs[i]] - int(not bool(idxs[i]))) for i in range(len(idxs))]

In [34]:
submit = np.stack([ids,dogs],axis=1)

In [35]:
np.savetxt(path + 'submission.csv', submit, fmt='%d,%.5f', header='id,label', comments='')
np.savetxt('submission.csv', submit, fmt='%d,%.5f', header='id,label', comments='')

In [36]:
from IPython.display import FileLink
FileLink('submission.csv')

/home/ubuntu/fastai-courses/deeplearning1/nbs/submission.csv